In [1]:
import numpy as np
from itertools import product
import math
import random
import copy

#### Test Metropolis

In [69]:
class Metropolis:
    def __init__(self, H):
        self.H = H
        # Possible values the spins can take
        self.spin_values = [-1,1]
        # Calculate the shape of the matrix and save useful variables
        self.num_spins = 1
        
    def metropolis_step(self):
        # Randomly select a spin (now it can only select 0)
        spin_to_flip = np.random.randint(self.num_spins)

        delta_energy = 2*(self.H[spin_to_flip]*self.sigma[spin_to_flip])

        p = math.exp(-delta_energy)
        r = random.random()

        #print(delta_energy, r, p)
        if delta_energy <= 0 or r<p:
            self.sigma[spin_to_flip] = -self.sigma[spin_to_flip]

    def metropolis_simulation(self, max_steps, burn_in_len=0):
        # Generate a random initial spin configuration
        self.sigma = random.choices(self.spin_values, k=self.num_spins)
        
        # Create a dictionary to count the frequencies
        samples_dict = {}
        for i in range(0, max_steps):
            # Take a metropolis step
            self.metropolis_step()
            
            # Save the state if the burn-in phase has passed
            if i >= burn_in_len:
                
                if not tuple(self.sigma) in samples_dict: # If the state is new, create an entry
                    samples_dict[tuple(self.sigma)] = 1
                else: # If the state already appeared, add one
                    samples_dict[tuple(self.sigma)] += 1

        # Normalize the probability
        sum_states = sum(samples_dict.values())
        normalized_dict = {k: v / sum_states for k, v in samples_dict.items()}
                                                 
        return normalized_dict

    def get_statistics(self, states_dict):
        total =  np.zeros(len(self.H))
        # Calculate the expectation
        for sigma, prob in states_dict.items():
            total += np.dot(sigma,prob)
            
        return total

    def test(self, simulation_len, burn_in_len):
        print('Initial H', self.H)
        print('Magnetization', np.tanh(self.H))
        
        samples_dict = self.metropolis_simulation(simulation_len, burn_in_len)
        
        print(samples_dict)

        # Calculate the data expectations        
        model_magnetization = self.get_statistics(samples_dict)
        
        
        print('Metropolis magnetization', model_magnetization)
        print('Diff', model_magnetization - np.tanh(self.H))

In [70]:
H = [0.70]
metropolis = Metropolis(H)

In [71]:
metropolis.test(simulation_len=1000, burn_in_len=100)

Initial H [0.7]
Magnetization [0.60436778]
{(1,): 0.8211111111111111, (-1,): 0.17888888888888888}
Metropolis magnetization [0.64222222]
Diff [0.03785445]


In [72]:
# Should be around 0 if random
acum = 0
for i in range(0,10000):   
    v = random.choices([-1, 1], k=1)
    acum += v[0]
acum

100

#### Only H

In [243]:
class SimpleIsingModel:
    def __init__(self, data, lr=0.0005):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        
        # Create a ndarray from the data
        self.data_activations_matrix = np.array(data)
        # Calculate the shape of the matrix and save useful variables
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]
        
        # Calculate the true data expectations
        # The mean
        self.data_mean = np.mean(self.data_activations_matrix, axis=0)
        
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        #self.H = np.sum(self.data_activations_matrix, axis=0)/ float(self.num_samples_data)
        # Try with random H
        self.H = np.random.normal(loc=0.0, scale=.01, size=self.num_spins)
        
        
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.0000005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            model_mean = np.zeros(self.H.shape)
            for sigma in product(self.spin_values, repeat=self.num_spins):
                # Calculate the expectation of the averages
                for i in range(0, self.num_spins):
                    model_mean[i] += sigma[i]*prob_dict[sigma]


            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            
            oldH = copy.deepcopy(self.H)
            # Take the step
            self.H = self.H + stepH
            
            # Calculate the variation of J for the termination condition
            totalParamVariation = np.sum(np.absolute(self.H - oldH))

            if epoch%100 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print(self.data_mean - model_mean, 'Diff')
                print()
            
            epoch += 1

        print(self.data_mean, 'Spins average')
        print(self.H, 'H')
    
    def metropolis_step(self):
        spin_to_flip = np.random.randint(self.num_spins)
        
        delta_energy = 2*(self.H[spin_to_flip]*self.sigma[spin_to_flip])
        
        p = math.exp(-delta_energy)
        r = random.random()
        
        if delta_energy <= 0 or r<p:
            
            self.sigma[spin_to_flip] = -self.sigma[spin_to_flip]
        
    def metropolis_simulation(self, max_steps, burn_in_len=0):
        # Generate a random initial spin configuration
        model_activations_list = []
        self.sigma = random.choices(self.spin_values, k=self.num_spins)
        
        for i in range(0, max_steps):
            
            # Take a metropolis step
            self.metropolis_step()
            
            # Save the state if the burn-in phase has passed
            if i > burn_in_len:
                model_activations_list.append(copy.deepcopy(self.sigma))

        
        return model_activations_list
    
    def train_metropolis(self, max_epochs = 300, simulation_len = 100, burn_in_len=0):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0000000005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:

            model_activations_list = self.metropolis_simulation(simulation_len, burn_in_len)   
            model_activations_matrix = np.array(model_activations_list)

            # Calculate the data expectations
            model_mean = np.mean(model_activations_matrix, axis=0)
            
            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            # Calculate the variation of H for the stop condition
            totalParamVariation = np.sum(np.absolute(self.H - oldH))
            
            if epoch > 9950:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print(self.data_mean - model_mean, 'Diff')
                print()

            epoch +=1
        
        print(self.data_mean, 'Spins average')
        print(self.H, 'H')


    def test_metropolis(self, simulation_len, burn_in_len):
        print('Initial H', self.H)
        print('Data average', self.data_mean)
        print('Magnetization', np.tanh(self.H))
        
        model_activations_list = self.metropolis_simulation(simulation_len, burn_in_len)
        model_activations_matrix = np.array(model_activations_list)

        # Calculate the data expectations
        model_mean = np.mean(model_activations_matrix, axis=0)
        
        print('Metropolis magnetization', model_mean)
        print('Diff', model_mean - np.tanh(self.H))


In [244]:
activations_list = [[1], [1], [-1], [-1], [1], [-1], [1], [1]]
s_ising = SimpleIsingModel(activations_list, lr=0.005)


In [242]:
s_ising.test_metropolis(simulation_len=1000, burn_in_len=100)

Initial H [0.00941252]
Data average [0.25]
Magnetization [0.00941224]
Metropolis magnetization [0.00556174]
Diff [-0.00385051]


In [ ]:
s_ising.train_metropolis( max_epochs = 10000, simulation_len = 1000, burn_in_len=100)

In [71]:
my_dict = {}
my_list = tuple([1,-1])
my_dict[my_list]=3

